In [ ]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [11]:
X, Y, n_values, indices_values = load_music_utils()
print('shape of X:', X.shape)
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('Shape of Y:', Y.shape)

shape of X: (60, 30, 78)
number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 78
Shape of Y: (30, 60, 78)


In [ ]:
n_a = 64 

In [12]:
reshapor = Reshape((1, 78))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

In [13]:
# GRADED FUNCTION: djmodel

def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    
    return model

In [14]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)

In [15]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [17]:
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
60/60 [==============================] - 12s - loss: 125.6465 - dense_2_loss_1: 4.3551 - dense_2_loss_2: 4.3437 - dense_2_loss_3: 4.3407 - dense_2_loss_4: 4.3478 - dense_2_loss_5: 4.3429 - dense_2_loss_6: 4.3343 - dense_2_loss_7: 4.3428 - dense_2_loss_8: 4.3319 - dense_2_loss_9: 4.3298 - dense_2_loss_10: 4.3324 - dense_2_loss_11: 4.3267 - dense_2_loss_12: 4.3284 - dense_2_loss_13: 4.3242 - dense_2_loss_14: 4.3161 - dense_2_loss_15: 4.3311 - dense_2_loss_16: 4.3223 - dense_2_loss_17: 4.3348 - dense_2_loss_18: 4.3355 - dense_2_loss_19: 4.3267 - dense_2_loss_20: 4.3387 - dense_2_loss_21: 4.3310 - dense_2_loss_22: 4.3296 - dense_2_loss_23: 4.3291 - dense_2_loss_24: 4.3305 - dense_2_loss_25: 4.3350 - dense_2_loss_26: 4.3212 - dense_2_loss_27: 4.3281 - dense_2_loss_28: 4.3258 - dense_2_loss_29: 4.3305 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.0000e+00 - dense_2_acc_2: 0.1000 - dense_2_acc_3: 0.0500 - dense_2_acc_4: 0.0167 - dense_2_acc_5: 0.0500 - dense_2_acc_6: 0.1000 - d

60/60 [==============================] - 1s - loss: 90.3438 - dense_2_loss_1: 4.1960 - dense_2_loss_2: 3.9330 - dense_2_loss_3: 3.5976 - dense_2_loss_4: 3.5460 - dense_2_loss_5: 3.3249 - dense_2_loss_6: 3.3837 - dense_2_loss_7: 3.3147 - dense_2_loss_8: 3.0050 - dense_2_loss_9: 3.1086 - dense_2_loss_10: 2.8883 - dense_2_loss_11: 3.0204 - dense_2_loss_12: 3.1242 - dense_2_loss_13: 2.8561 - dense_2_loss_14: 2.8462 - dense_2_loss_15: 2.8959 - dense_2_loss_16: 3.0986 - dense_2_loss_17: 2.9345 - dense_2_loss_18: 2.9229 - dense_2_loss_19: 2.8194 - dense_2_loss_20: 3.0334 - dense_2_loss_21: 2.9594 - dense_2_loss_22: 2.8793 - dense_2_loss_23: 2.9958 - dense_2_loss_24: 2.9049 - dense_2_loss_25: 3.1556 - dense_2_loss_26: 2.7177 - dense_2_loss_27: 2.9568 - dense_2_loss_28: 2.9505 - dense_2_loss_29: 2.9743 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.0667 - dense_2_acc_2: 0.2167 - dense_2_acc_3: 0.2167 - dense_2_acc_4: 0.1667 - dense_2_acc_5: 0.2833 - dense_2_acc_6: 0.1167 - dense_2_acc_7: 0.10

60/60 [==============================] - 1s - loss: 56.8619 - dense_2_loss_1: 4.1083 - dense_2_loss_2: 3.5162 - dense_2_loss_3: 2.8204 - dense_2_loss_4: 2.5524 - dense_2_loss_5: 2.2282 - dense_2_loss_6: 2.0731 - dense_2_loss_7: 1.9801 - dense_2_loss_8: 1.7936 - dense_2_loss_9: 1.8993 - dense_2_loss_10: 1.8263 - dense_2_loss_11: 1.8381 - dense_2_loss_12: 1.8056 - dense_2_loss_13: 1.5177 - dense_2_loss_14: 1.6020 - dense_2_loss_15: 1.7273 - dense_2_loss_16: 1.7522 - dense_2_loss_17: 1.6626 - dense_2_loss_18: 1.7066 - dense_2_loss_19: 1.6317 - dense_2_loss_20: 1.6950 - dense_2_loss_21: 1.7697 - dense_2_loss_22: 1.6790 - dense_2_loss_23: 1.6914 - dense_2_loss_24: 1.6616 - dense_2_loss_25: 1.7675 - dense_2_loss_26: 1.5971 - dense_2_loss_27: 1.7233 - dense_2_loss_28: 1.6303 - dense_2_loss_29: 1.6053 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.1000 - dense_2_acc_2: 0.3000 - dense_2_acc_3: 0.3833 - dense_2_acc_4: 0.2333 - dense_2_acc_5: 0.3500 - dense_2_acc_6: 0.3833 - dense_2_acc_7: 0.43

60/60 [==============================] - 1s - loss: 31.0294 - dense_2_loss_1: 4.0153 - dense_2_loss_2: 2.9875 - dense_2_loss_3: 1.8805 - dense_2_loss_4: 1.5455 - dense_2_loss_5: 1.2705 - dense_2_loss_6: 1.0680 - dense_2_loss_7: 0.8744 - dense_2_loss_8: 0.9031 - dense_2_loss_9: 0.8956 - dense_2_loss_10: 0.7941 - dense_2_loss_11: 0.8383 - dense_2_loss_12: 0.8211 - dense_2_loss_13: 0.6825 - dense_2_loss_14: 0.7208 - dense_2_loss_15: 0.7496 - dense_2_loss_16: 0.8202 - dense_2_loss_17: 0.7311 - dense_2_loss_18: 0.7475 - dense_2_loss_19: 0.7390 - dense_2_loss_20: 0.8252 - dense_2_loss_21: 0.8179 - dense_2_loss_22: 0.7976 - dense_2_loss_23: 0.8094 - dense_2_loss_24: 0.7830 - dense_2_loss_25: 0.7773 - dense_2_loss_26: 0.8043 - dense_2_loss_27: 0.7962 - dense_2_loss_28: 0.7332 - dense_2_loss_29: 0.8004 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.1000 - dense_2_acc_2: 0.4167 - dense_2_acc_3: 0.5833 - dense_2_acc_4: 0.5500 - dense_2_acc_5: 0.6833 - dense_2_acc_6: 0.8667 - dense_2_acc_7: 0.95

60/60 [==============================] - 1s - loss: 16.6870 - dense_2_loss_1: 3.9472 - dense_2_loss_2: 2.4741 - dense_2_loss_3: 1.2170 - dense_2_loss_4: 0.7883 - dense_2_loss_5: 0.6233 - dense_2_loss_6: 0.4860 - dense_2_loss_7: 0.3570 - dense_2_loss_8: 0.3721 - dense_2_loss_9: 0.3539 - dense_2_loss_10: 0.3073 - dense_2_loss_11: 0.3335 - dense_2_loss_12: 0.3213 - dense_2_loss_13: 0.2698 - dense_2_loss_14: 0.2696 - dense_2_loss_15: 0.2808 - dense_2_loss_16: 0.2930 - dense_2_loss_17: 0.2846 - dense_2_loss_18: 0.2852 - dense_2_loss_19: 0.3084 - dense_2_loss_20: 0.3156 - dense_2_loss_21: 0.3346 - dense_2_loss_22: 0.3103 - dense_2_loss_23: 0.3058 - dense_2_loss_24: 0.2979 - dense_2_loss_25: 0.3064 - dense_2_loss_26: 0.2998 - dense_2_loss_27: 0.3018 - dense_2_loss_28: 0.2848 - dense_2_loss_29: 0.3574 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.1000 - dense_2_acc_2: 0.4167 - dense_2_acc_3: 0.7000 - dense_2_acc_4: 0.8500 - dense_2_acc_5: 0.8833 - dense_2_acc_6: 0.9500 - dense_2_acc_7: 0.98

60/60 [==============================] - 1s - loss: 10.8439 - dense_2_loss_1: 3.8981 - dense_2_loss_2: 2.0568 - dense_2_loss_3: 0.8226 - dense_2_loss_4: 0.3898 - dense_2_loss_5: 0.3158 - dense_2_loss_6: 0.2335 - dense_2_loss_7: 0.1583 - dense_2_loss_8: 0.1704 - dense_2_loss_9: 0.1515 - dense_2_loss_10: 0.1288 - dense_2_loss_11: 0.1455 - dense_2_loss_12: 0.1408 - dense_2_loss_13: 0.1212 - dense_2_loss_14: 0.1180 - dense_2_loss_15: 0.1248 - dense_2_loss_16: 0.1262 - dense_2_loss_17: 0.1197 - dense_2_loss_18: 0.1268 - dense_2_loss_19: 0.1353 - dense_2_loss_20: 0.1395 - dense_2_loss_21: 0.1475 - dense_2_loss_22: 0.1349 - dense_2_loss_23: 0.1313 - dense_2_loss_24: 0.1301 - dense_2_loss_25: 0.1331 - dense_2_loss_26: 0.1274 - dense_2_loss_27: 0.1310 - dense_2_loss_28: 0.1291 - dense_2_loss_29: 0.1562 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.1000 - dense_2_acc_2: 0.4833 - dense_2_acc_3: 0.7833 - dense_2_acc_4: 0.9833 - dense_2_acc_5: 0.9667 - dense_2_acc_6: 0.9833 - dense_2_acc_7: 1.00

60/60 [==============================] - 1s - loss: 8.4746 - dense_2_loss_1: 3.8578 - dense_2_loss_2: 1.7603 - dense_2_loss_3: 0.6002 - dense_2_loss_4: 0.2341 - dense_2_loss_5: 0.1822 - dense_2_loss_6: 0.1318 - dense_2_loss_7: 0.0889 - dense_2_loss_8: 0.0973 - dense_2_loss_9: 0.0821 - dense_2_loss_10: 0.0701 - dense_2_loss_11: 0.0796 - dense_2_loss_12: 0.0748 - dense_2_loss_13: 0.0681 - dense_2_loss_14: 0.0663 - dense_2_loss_15: 0.0701 - dense_2_loss_16: 0.0686 - dense_2_loss_17: 0.0640 - dense_2_loss_18: 0.0708 - dense_2_loss_19: 0.0742 - dense_2_loss_20: 0.0753 - dense_2_loss_21: 0.0797 - dense_2_loss_22: 0.0726 - dense_2_loss_23: 0.0702 - dense_2_loss_24: 0.0683 - dense_2_loss_25: 0.0717 - dense_2_loss_26: 0.0694 - dense_2_loss_27: 0.0729 - dense_2_loss_28: 0.0713 - dense_2_loss_29: 0.0818 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.1000 - dense_2_acc_2: 0.5333 - dense_2_acc_3: 0.8667 - dense_2_acc_4: 0.9833 - dense_2_acc_5: 0.9833 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.000

60/60 [==============================] - 1s - loss: 7.3026 - dense_2_loss_1: 3.8213 - dense_2_loss_2: 1.5483 - dense_2_loss_3: 0.4631 - dense_2_loss_4: 0.1606 - dense_2_loss_5: 0.1140 - dense_2_loss_6: 0.0837 - dense_2_loss_7: 0.0596 - dense_2_loss_8: 0.0636 - dense_2_loss_9: 0.0533 - dense_2_loss_10: 0.0452 - dense_2_loss_11: 0.0516 - dense_2_loss_12: 0.0485 - dense_2_loss_13: 0.0440 - dense_2_loss_14: 0.0440 - dense_2_loss_15: 0.0456 - dense_2_loss_16: 0.0451 - dense_2_loss_17: 0.0419 - dense_2_loss_18: 0.0457 - dense_2_loss_19: 0.0479 - dense_2_loss_20: 0.0481 - dense_2_loss_21: 0.0505 - dense_2_loss_22: 0.0476 - dense_2_loss_23: 0.0454 - dense_2_loss_24: 0.0442 - dense_2_loss_25: 0.0463 - dense_2_loss_26: 0.0454 - dense_2_loss_27: 0.0476 - dense_2_loss_28: 0.0468 - dense_2_loss_29: 0.0538 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.1000 - dense_2_acc_2: 0.5833 - dense_2_acc_3: 0.8833 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.000

60/60 [==============================] - 1s - loss: 6.6144 - dense_2_loss_1: 3.7880 - dense_2_loss_2: 1.3898 - dense_2_loss_3: 0.3701 - dense_2_loss_4: 0.1216 - dense_2_loss_5: 0.0782 - dense_2_loss_6: 0.0608 - dense_2_loss_7: 0.0444 - dense_2_loss_8: 0.0460 - dense_2_loss_9: 0.0390 - dense_2_loss_10: 0.0328 - dense_2_loss_11: 0.0369 - dense_2_loss_12: 0.0345 - dense_2_loss_13: 0.0317 - dense_2_loss_14: 0.0320 - dense_2_loss_15: 0.0333 - dense_2_loss_16: 0.0330 - dense_2_loss_17: 0.0302 - dense_2_loss_18: 0.0329 - dense_2_loss_19: 0.0347 - dense_2_loss_20: 0.0345 - dense_2_loss_21: 0.0359 - dense_2_loss_22: 0.0345 - dense_2_loss_23: 0.0325 - dense_2_loss_24: 0.0322 - dense_2_loss_25: 0.0338 - dense_2_loss_26: 0.0333 - dense_2_loss_27: 0.0344 - dense_2_loss_28: 0.0342 - dense_2_loss_29: 0.0392 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.1000 - dense_2_acc_2: 0.6000 - dense_2_acc_3: 0.9000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.000

60/60 [==============================] - 1s - loss: 6.1601 - dense_2_loss_1: 3.7575 - dense_2_loss_2: 1.2628 - dense_2_loss_3: 0.3074 - dense_2_loss_4: 0.0978 - dense_2_loss_5: 0.0602 - dense_2_loss_6: 0.0476 - dense_2_loss_7: 0.0353 - dense_2_loss_8: 0.0357 - dense_2_loss_9: 0.0305 - dense_2_loss_10: 0.0254 - dense_2_loss_11: 0.0286 - dense_2_loss_12: 0.0264 - dense_2_loss_13: 0.0246 - dense_2_loss_14: 0.0251 - dense_2_loss_15: 0.0260 - dense_2_loss_16: 0.0260 - dense_2_loss_17: 0.0233 - dense_2_loss_18: 0.0254 - dense_2_loss_19: 0.0269 - dense_2_loss_20: 0.0266 - dense_2_loss_21: 0.0277 - dense_2_loss_22: 0.0267 - dense_2_loss_23: 0.0250 - dense_2_loss_24: 0.0251 - dense_2_loss_25: 0.0262 - dense_2_loss_26: 0.0259 - dense_2_loss_27: 0.0269 - dense_2_loss_28: 0.0270 - dense_2_loss_29: 0.0303 - dense_2_loss_30: 0.0000e+00 - dense_2_acc_1: 0.1000 - dense_2_acc_2: 0.6167 - dense_2_acc_3: 0.9167 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.000

In [18]:
# GRADED FUNCTION: music_inference_model

def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = []

    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x,initial_state=[a,c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        x = Lambda(one_hot)(out)
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs=[x0, a0, c0], outputs=outputs)
        
    return inference_model

In [19]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

In [20]:
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

In [21]:
# GRADED FUNCTION: predict_and_sample

def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = np.argmax(pred, axis=-1)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (1, )
    results = to_categorical(indices, num_classes=78)
    
    return results, indices

In [22]:
results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

np.argmax(results[12]) = 32
np.argmax(results[17]) = 9
list(indices[12:18]) = [array([32]), array([9]), array([2]), array([76]), array([32]), array([9])]


In [ ]:
out_stream = generate_music(inference_model)

In [ ]:
IPython.display.Audio('./data/30s_trained_model.mp3')